In [1]:
import pandas as pd
import numpy as np
import math 

inputs wind and direction, outputs adjusted azimuth & distance<br>
given impact point of wind, calculate implied wind distance & azimuth<br>
calculate length of third side given TSG angle and lengths of the two other sides (TSG = Target, Spotter, Gunner)

In [2]:

def findTSGAngle(spotterToTargetAzimuth, spotterToGunAzimuth):
    azimuthArray = [spotterToTargetAzimuth, spotterToGunAzimuth]
    aziMin = min(azimuthArray)
    aziMax = max(azimuthArray)    


    if (360 - aziMax - aziMin) > 0:
        return min(aziMax - aziMin, 360 - aziMax - aziMin)
    else:
        return (aziMax - aziMin)
    

# print(findTSGAngle(0,90))
# print(findTSGAngle(264,171))


In [3]:
def findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    dST = spotterToTargetDistance
    dSG = spotterToGunDistance
    aTSG = findTSGAngle(spotterToTargetAzimuth,spotterToGunAzimuth)

    # Law of Cosines to solve 3rd unknown angle
    # a^2 = b^2 + c^2 − 2*b*c*cos(A)

    # math.radians() inputs degrees, outputs radians
    # math.cos() inputs radians, outputs radians
    # math.degrees inputs radians, outputs degrees


    distGunToTarget = math.sqrt(dST**2 + dSG**2 - 2*dST*dSG*math.cos(math.radians(aTSG)))

    return distGunToTarget

# print(findDistanceGunToTarget(253, 66, 173, 16))

In [4]:
def findTGSAngle(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    # https://courses.lumenlearning.com/suny-osalgebratrig/chapter/non-right-triangles-law-of-cosines/ intuition
    # reference figure 3
    # distance GT = a
    # distance ST = b
    # distance SG = c
    # angle TSG = alpha
    # objective: find angle Beta
    # use equation cos(beta) = (a**2 + c**2 - b**2)/(2*a*c)

    dGT = findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    dST = spotterToTargetDistance
    dSG = spotterToGunDistance

    # math.acos() inputs radians & outputs radians
    aTGS = math.degrees(math.acos((dGT**2 + dSG**2 - dST**2)/(2*dGT*dSG)))
    return aTGS

# print(findTGSAngle(0,3,90,4))


In [5]:
def findAzimuthGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    aTGS = findTGSAngle(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    aTSG = findTSGAngle(spotterToTargetAzimuth, spotterToGunAzimuth)
    aSTG = 180 - (aTGS + aTSG)
    aziSG = spotterToGunAzimuth
    aziST = spotterToTargetAzimuth


    # Create imaginary box around 3 points (Spotter-Target-Gun) box & try to use alternate interior angle theorem

    # Graph 1
    # if aziSG > 180, take aziSG - 180
    # SG > ST
    # ST < 180
    # SG > 180
    # (aziSG - 180) + aTGS

    # Graph 2
    # if aziSG < 180, take aziSG + 180
    # SG > ST
    # ST < 180
    # SG < 180
    # (aziSG + 180) + aTGS

    # Graph 3
    # SG < ST
    # ST > 180
    # SG > 180
    # (aziSG - 180) - aTGS

    # Graph 4
    # SG < ST
    # ST < 180
    # SG < 180
    # same formula as graph 2
    # (aziSG + 180) - aTGS

    # if SG > 180, first term is subtraction
    # if SG < 180, first term is addition

    # if SG > ST, second term is addition
    # if SG < ST, second term is subtraction
    if (aziSG > aziST) and (aziSG > 180):
        return (aziSG - 180) + aTGS

    elif (aziSG > aziST) and (aziSG < 180):
        return (aziSG + 180) + aTGS
    
    elif (aziSG < aziST) and (aziSG > 180):
        return (aziSG - 180) - aTGS

    elif (aziSG < aziST) and (aziSG < 180):
        return (aziSG + 180) - aTGS
        
print(findAzimuthGunToTarget(253, 66, 173, 16))

266.99488180869963


In [6]:
# def findWindAdjustedGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce):
#     # https://www.omnicalculator.com/physics/wind-correction-angle
#     # heading: "How to calculate wind correction angle"
#     # adjust windAzimuth if greater than 90 degrees

#     # tier 3 wind pushes azimuth 10 if perpendicular
#     # distance doesnt change
#     # 580 AT 213.2
#     # SOUTHEASTERNL
#     # 580>604, AZIMUTH 213-224, 10 DEGREE SHIFT

#     # if windAzimuth > 0 && windAzimuth < 90
#     # cos(windAzimuth) = (bx / windForce)

#     # alpha or course
#     desiredBulletCourse = unadjustedGunToTargetAzimuth

#     # in article referred to as "True airspeed", or TAS
#     bulletAirSpeed = 0

#     # wind angle = the angle between the wind direction and the desired course, delta in article
#     windAngleAzimuth = desiredBulletCourse - windAzimuth


#     windCorrectionAzimuth = math.degrees(math.asin(windForce / bulletAirSpeed * math.degrees(math.sin(math.radians(windAngleAzimuth)))))

#     adjustedGunToTargetAzimuth = desiredBulletCourse + windCorrectionAzimuth

#     return adjustedGunToTargetAzimuth


def findWindAdjustedGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce):
    # https://nigelef.tripod.com/fc_ballistics.htm
    # https://youtu.be/p026Y0WUy6Q?t=364 used this one

    # windAzimuth input is supposed to be with compass oriented to the North, and calculating azimuth from the way wind is blowing
    # e.g. windAzimuth of 180 degrees is blowing the flag from north to south

    # original distance is in meters
    unadjustedGunToTargetDistanceYards = unadjustedGunToTargetDistance * 1.09361

    #some calculation to turn windForce, which is a number 1-5, to windMPH
    # NOTE: NEED TO FIND CORRECT FORMULA *************
    windSpeedMPH = windForce 

    # example in video for .308 is 100-500yd = 13, 600yd = 12, 700-800yd = 13, 900yd = 10
    #conversion to get ballistic constant as it changes with different distances
    # NOTE: NEED TO FIND CORRECT FORMULA *************
    ballisticConstant = unadjustedGunToTargetDistanceYards 

    #windAdjustment is in minutes of angle
    # 0.0166667/1 = degree/MOA
    # FOR FULL-VALUE WIND, or wind is perpendicular to ideal bullet path
    windAdjustmentInMOA = ((unadjustedGunToTargetDistanceYards/100)*windSpeedMPH)/ballisticConstant 
    windAdjustmentInAzimuth = windAdjustmentInMOA * 0.0166667

    # inches per minute of angle = unadjustedGunToTargetDistanceYards/100
    # 0.0254/1 = meter/inch
    # 60/1 = MOA/degree
    metersPerMOA = (unadjustedGunToTargetDistanceYards/100 * windAdjustmentInMOA)*0.0254
    metersPerDegree = metersPerMOA * 60

    # full-value wind, half-value, etc 
    # https://youtu.be/p026Y0WUy6Q?t=674
    # intuition is unadjustedGunToTargetAzimuth is set to 0, for perspective on wind affect on bullet course
    if (windAzimuth - unadjustedGunToTargetAzimuth < 0):
        windAzimuthRelativeToGunToTarget = windAzimuth - unadjustedGunToTargetAzimuth
    else:
        windAzimuthRelativeToGunToTarget = windAzimuth - unadjustedGunToTargetAzimuth + 360

    # used clock method to calculate wind-adjustment
    # https://www.bevfitchett.us/rifle-marksmanship-m16a1/effects-of-wind-and-gravity.html
    # full value = 90, 9/10 value = 65, 3/4 value = 45, 1/2 value = 30, 1/4 value = 15
    # https://www.rifleshootermag.com/editorial/shooting-tips-determing-wind-values/83917 
    
    # note length of degreeArray must match valueArray, basically dictionary
    degreeArray = [90, 65, 45, 30, 15, 0]
    valueArray = [1, 9/10, 3/4, 1/2, 1/4, 0]

    # helper function to find nearest degree value
    def find_nearest(array,value):
        idx = np.searchsorted(array, value, side="left")
        if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
            return idx-1
        else:
            return idx
            
    # determines strength of wind on pushing azimuth left & right
    if (windAzimuthRelativeToGunToTarget % 90 == 0) or (windAzimuthRelativeToGunToTarget % 90 == 2): #quadrants 1 & 3
        index = find_nearest(degreeArray ,windAzimuthRelativeToGunToTarget - (windAzimuthRelativeToGunToTarget % 90)*90)
        windDirectionMultiplierForAzimuth = valueArray[index]
    elif (windAzimuthRelativeToGunToTarget % 90 == 1) or (windAzimuthRelativeToGunToTarget % 90 == 3): #quadrants 2 & 4
        index = find_nearest(degreeArray ,windAzimuthRelativeToGunToTarget - (windAzimuthRelativeToGunToTarget % 90)*90)
        windDirectionMultiplierForAzimuth = valueArray[len(degreeArray) - index - 1]
    
    # if wind blows from left, increase azimuth
    # if wind blows from right, decrease azimuth
    if (windAzimuthRelativeToGunToTarget % 180 == 1): #quadrants 1 & 4
        # wind blows from right to left if x % 180 == 0
        windBlowsFromRightOrLeft = -1
    elif (windAzimuthRelativeToGunToTarget % 180 == 0): #quadrants 2 & 3
        # wind blows from left to right if x % 180 == 1
        windBlowsFromRightOrLeft = 1

    adjustedForWindDirectionWindAdjustmentInAzimuth = windDirectionMultiplierForAzimuth * windAdjustmentInAzimuth * windBlowsFromRightOrLeft

    # this is azimuth adjustment for lateral drift
    adjustedGunToTargetAzimuth = unadjustedGunToTargetAzimuth + adjustedForWindDirectionWindAdjustmentInAzimuth

    # note: need to eventually figure out how wind affects distance
    # determines strength of wind on pushing distance far or close
    if (windAzimuthRelativeToGunToTarget % 90 == 1) or (windAzimuthRelativeToGunToTarget % 90 == 3): #quadrants 2 & 4
        index = find_nearest(degreeArray ,windAzimuthRelativeToGunToTarget - (windAzimuthRelativeToGunToTarget % 90)*90)
        windDirectionMultiplierForDistance = valueArray[index]
    elif (windAzimuthRelativeToGunToTarget % 90 == 0) or (windAzimuthRelativeToGunToTarget % 90 == 2): #quadrants 1 & 3
        index = find_nearest(degreeArray ,windAzimuthRelativeToGunToTarget - (windAzimuthRelativeToGunToTarget % 90)*90)
        windDirectionMultiplierForDistance = valueArray[len(degreeArray) - index - 1]

    # if wind blows from north to south, increase distance
    # if wind blows from south to north, decrease distance
    if (windAzimuthRelativeToGunToTarget % 90 == 3) or (windAzimuthRelativeToGunToTarget % 90 == 0): #quadrants 1 & 2
        # wind blows from south to north
        windBlowsFromNorthOrSouth = -1
    elif (windAzimuthRelativeToGunToTarget % 90 == 1) or (windAzimuthRelativeToGunToTarget % 90 == 2): #quadrants 3 & 4
        # wind blows from north to south
        windBlowsFromNorthOrSouth = 1

    changeInDistanceDueToWind = metersPerDegree * windDirectionMultiplierForDistance * windBlowsFromNorthOrSouth
    adjustedGunToTargetDistance = unadjustedGunToTargetDistance + changeInDistanceDueToWind * windAdjustmentInAzimuth
    
    return [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, adjustedGunToTargetAzimuth, adjustedGunToTargetDistance]


In [7]:
def comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce):
    unadjustedGunToTargetDistance = findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    unadjustedGunToTargetAzimuth = findAzimuthGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)

    # for when I figure out model
    # return findWindAdjustedGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce)
    return [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance]

# print(findDistanceGunToTarget(254, 65, 173, 17))
# print(findAzimuthGunToTarget(254, 65, 173, 17)) ************ test here

In [8]:
# columnNames = ["spotterToTargetAzimuth", "spotterToTargetDistance", "spotterToGunAzimuth", "spotterToGunDistance", "windAzimuth", "windForce", "unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance", "adjustedGunToTargetAzimuth","adjustedGunToTargetDistance"]
# firingHistoryArray = pd.DataFrame(columns = columnNames)
# print(firingHistoryArray)

In [9]:
def artilleryContinuous(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, firingHistoryArray):
    while True:
        firingHistoryArray.tail(5)
        userInput = input("Enter <Stop> to exit or press <Esc> key to continute. \n").lower()
        if userInput == "stop":
            print("Exiting..")
            break
        gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        firingHistoryArray = firingHistoryArray.append([spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals)
        firingHistoryArray.tail(1)


In [10]:
# The plan
# Purpose: build model to predict gun azi/dist given gunAzimuth, gunDistance, windAzimuth, & windForce (multi-variate?)
# Data to work with: spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, spotterToImpactAzimuth, spotterToImpactDistance, windAzimuth, windForce

# Step 0: get data from training island by firing artillery shills, adapt artilleryContinuous in order to generate data (perhaps read & write from excel file function?)

# Step 1: make model to predict impact azimuth and impact distance
# implementation of step 1:
    # model 1: gunToImpactAzimuth:
        # dependent variables: gunToTargetDistance, windAzimuth, windForce
        # independent variable: gunToImpactAzimuth

    # model 2: gunToImpactDistance:
        # dependent variables: gunToTargetDistance, windAzimuth, windForce
        # independent variable: gunToImpactDistance


# Step 2: use goal seek to find necessary gunToTargetAzimuth and gunToTargetDistance to make impact azimuth and impact distance equal to target azimuth and target distance, respectively
# implemention of step 2:
    # model to calculate newGunToTargetAzimuth to hit target
    # Input:
    # Output: the necessary azimuth in order to get impact = target

    # model to calculate newGunToTargetDistance to hit target
    # Input: 
    # Output: the necessary distance in order to get impact = target

# model
    # indepentdent variable =  distance from gun to target
    # dependent variable = difference between predicted impact and actual impact
    # use to find how error changes over distance fired
    # use to find given error at certain distance
    # error that cannot be explained by wind

In [15]:
# https://openpyxl.readthedocs.io/en/stable/

from os.path import exists
from openpyxl import Workbook
import pandas as pd

def getDataForArtilleryModels(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon):
    firingHistoryArray = pd.DataFrame(columns=list(["spotterToTargetAzimuth", "spotterToTargetDistance", "spotterToGunAzimuth", "spotterToGunDistance", 
        "windAzimuth", "windForce", "unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance", "gunToImpactAzimuth", "gunToImpactDistance"]))
    print("Keep in mind multiple input responses should be space separated.")

    while True:
        print(firingHistoryArray.tail(2))
        print("\n\n")
        userInput = input("Enter <S> to stop, <C> to change variables, or press <Esc> key to continue. \t").lower() # potentially add another option to change values in dataframe if a mistake was entered

        # saves current data in dataframe to excel file & closes program
        if userInput == "s":
            # if .xlsx file already exists
            if (exists(f"{excelFileName}.xlsx")):
                # add firingHistoryArray dataframe to excel file to save data
                # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html   pd.ExcelWriter section
                # appends firingHistoryArray to existing excel file named whatever the <excelFilName> was, MUST BE IN SAME DIRECTORY AS PROGRAM

                # import excel file, converts to pandas dataframe
                xlToPandas = pd.ExcelFile(f"{excelFileName}.xlsx")
                dataFrameFromPastExcel = xlToPandas.parse(f"{caliberOfWeapon}")
                dataFrameFromPastExcel.reset_index(drop=True, inplace=True)

                # makes the index of the two merge and go from 0 to N, vs restarting from zero
                dataFrameFromPastExcel.reset_index(drop=True, inplace=True)
                firingHistoryArray.reset_index(drop=True, inplace=True)

                # merge historical excel data with new data to upload to Excel file
                combinedDataframe = pd.concat([dataFrameFromPastExcel, firingHistoryArray], ignore_index=True)
                with pd.ExcelWriter(f"{excelFileName}.xlsx", mode = "a", engine="openpyxl", if_sheet_exists="replace") as writer:
                    combinedDataframe.to_excel(writer, sheet_name = f"{caliberOfWeapon}", header="None")
            else:
                # create new .xlsx file and save firingHistoryArray dataframe to it and save file if doesn't already exist
                # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
                firingHistoryArray.to_excel(f"{excelFileName}.xlsx", sheet_name=caliberOfWeapon)
            print("Exiting..")
            break



        # changes inputs if inputs change, ask user if these are the correct inputs before applying
        elif (userInput == "c"):
            # display possible inputs
            # changes inputs to what the other puts in
            while True:
                currentVariableValuesArray = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce]
                print(currentVariableValuesArray)
                variableNumberAndValueToChange = int(input("Enter number 1 through 6 to change the corresponding variable between: spotterToTargetAzimuth(1), spotterToTargetDistance(2), spotterToGunAzimuth(3), spotterToGunDistance(4), windAzimuth(5), windForce(6) \n," + 
                    "and the value to change it to with a space between. Press <Esc> to leave. \t").split())

                if variableNumberAndValueToChange == "":
                    break

                # changes variable to input
                currentVariableValuesArray[int(variableNumberAndValueToChange[0])] = int(variableNumberAndValueToChange[1])

                # assigns values in array to variables 
                spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce = currentVariableValuesArray.split()

                # confirm these are the correct variables?
                print(currentVariableValuesArray)
                userInputToLeaveChange = input("Are these the correct variables? Enter <Y> to leave. \t").lower()
                if userInputToLeaveChange == "y":
                    break

        # once impact hits, ask for impact azimuth and impact distance
        while True:
            impactAzi, impactDist = input("Please enter impact azimuth & impact distance, i.e. spotterToImpactAzimuth & spotterToImpactDistance \t").split()
            print(f"Impact Azimuth: {impactAzi} \n Impact Distance: {impactDist}")
            areImpactCorrectInput = input("Are these impact values correct? Enter <Y> to leave loop.").lower()
            if areImpactCorrectInput == "y":
                break

        impactValuesSpotterToImpact = [int(impactAzi), int(impactDist)]
        impactValuesGunToImpact = [findAzimuthGunToTarget(impactValuesSpotterToImpact[0], impactValuesSpotterToImpact[1], spotterToGunAzimuth, spotterToGunDistance), findDistanceGunToTarget(impactValuesSpotterToImpact[0], impactValuesSpotterToImpact[1], spotterToGunAzimuth, spotterToGunDistance)]
        
        # for when I have a model to calculate adjustments for wind
        # gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        # firingHistoryArray = firingHistoryArray.append([spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact)

        gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        valuesToStoreInDataframe = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact
        firingHistoryArrayLength = len(firingHistoryArray)
        firingHistoryArray.reset_index(drop=True, inplace=True)
        firingHistoryArray.loc[firingHistoryArrayLength] = valuesToStoreInDataframe


# input variables
# (spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon)

getDataForArtilleryModels(254,65,173,17,124,4,"artilleryData","120mm")


Keep in mind multiple input responses should be space separated.
Empty DataFrame
Columns: [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, gunToImpactAzimuth, gunToImpactDistance]
Index: []



Impact Azimuth: 500 
 Impact Distance: 50
   spotterToTargetAzimuth  spotterToTargetDistance  spotterToGunAzimuth  \
0                   254.0                     65.0                173.0   

   spotterToGunDistance  windAzimuth  windForce  unadjustedGunToTargetAzimuth  \
0                  17.0        124.0        4.0                    269.074221   

   unadjustedGunToTargetDistance  gunToImpactAzimuth  gunToImpactDistance  
0                      64.562217          220.522834            36.922351  



Impact Azimuth: 200 
 Impact Distance: 20
   spotterToTargetAzimuth  spotterToTargetDistance  spotterToGunAzimuth  \
0                   254.0                     65.0

In [12]:
fish = "txt"
x=1
while x < 4:
    if x == 2:
        fish="hehe"
    print(fish)
    x+=1


txt
hehe
hehe
